In [1]:
import pandas as pd

In [5]:
Republican  = pd.read_csv('joint_republican.csv')
Republican.drop(['Unnamed: 0'], axis=1, inplace  = True)
Democrats  = pd.read_csv('joint_democrats.csv')
Democrats.drop(['Unnamed: 0'], axis=1, inplace  = True)

In [6]:
Final_RD_Tweet_User = pd.concat([Democrats, Republican], axis=0, ignore_index=True)

In [8]:
Final_RD_Tweet_User.to_csv('Final_RD_Tweet_User.csv')

In [10]:
Final_RD_Tweet_User.head(3)

,username,tweet,Target
0,benniegthompson,"['today', 'rememb', 'reflect', 'life', 'civil'...",D
1,bettymccollum04,"['state', 'foreign', 'oper', 'appropsdem', 'bi...",D
2,billpascrell,"[""nation'"", 'firefight', 'put', 'live', 'line'...",D


In [78]:
Final_RD_Tweet_User.set_index('username', inplace = True)

In [79]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn import metrics
import numpy as np
import xgboost as xgb
import os
os.environ['KMP_DUPLICATE_LIB_OK']= 'True'

In [80]:
data = Final_RD_Tweet_User['tweet']
target = Final_RD_Tweet_User['Target']

In [81]:
X_train, X_test, y_train, y_test = train_test_split(data , target, test_size=0.20)

In [82]:
vectorizer = TfidfVectorizer()
tf_idf_data_train = vectorizer.fit_transform(X_train)
tf_idf_data_test = vectorizer.transform(X_test)

In [83]:
tf_idf_data_train.shape

(406, 9516)

In [84]:
nb_classifier = MultinomialNB()
rf_classifier = RandomForestClassifier( verbose=1,n_jobs=-1)
xgb_classifier = xgb.XGBClassifier()

## Naive Base

In [85]:
nb_classifier.fit(tf_idf_data_train, y_train)
nb_train_preds = nb_classifier.predict(tf_idf_data_train)
nb_test_preds = nb_classifier.predict(tf_idf_data_test)

In [86]:
nb_train_score = accuracy_score(y_train, nb_train_preds)
nb_test_score = accuracy_score(y_test, nb_test_preds)
print("Training Accuracy: {:.4} \t\t Testing Accuracy: {:.4}".format(nb_train_score, nb_test_score))
print("")
print('-'*70)

Training Accuracy: 0.9064 		 Testing Accuracy: 0.7353

----------------------------------------------------------------------


In [87]:
confusion = metrics.confusion_matrix(y_test, nb_test_preds, labels = ['D','R'])
confusion

array([[50,  2],
       [25, 25]])

## Random Forest

In [88]:
rf_classifier.fit(tf_idf_data_train, y_train)
rf_train_preds = rf_classifier.predict(tf_idf_data_train)
rf_test_preds = rf_classifier.predict(tf_idf_data_test)

/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished


In [89]:
rf_train_score = accuracy_score(y_train, rf_train_preds)
rf_test_score = accuracy_score(y_test, rf_test_preds)
print('Random Forest')
print("Training Accuracy: {:.4} \t\t Testing Accuracy: {:.4}".format(rf_train_score, rf_test_score))

Random Forest
Training Accuracy: 0.9926 		 Testing Accuracy: 0.9314


In [90]:
confusion = metrics.confusion_matrix(y_test, rf_test_preds, labels = ['D','R'])
confusion

array([[48,  4],
       [ 3, 47]])

In [91]:
#finding where the mistake was

## xgboost

In [92]:
xgb_classifier.fit(tf_idf_data_train, y_train)
xgb_train_preds = xgb_classifier.predict(tf_idf_data_train)
xgb_test_preds = xgb_classifier.predict(tf_idf_data_test)

In [93]:
xgb_train_score = accuracy_score(y_train, xgb_train_preds)
xgb_test_score = accuracy_score(y_test, xgb_test_preds)
print('xgboost')
print("Training Accuracy: {:.4} \t\t Testing Accuracy: {:.4}".format(xgb_train_score, xgb_test_score))

xgboost
Training Accuracy: 1.0 		 Testing Accuracy: 0.9706


In [94]:
confusion = metrics.confusion_matrix(y_test, xgb_test_preds, labels = ['D','R'])
confusion

array([[50,  2],
       [ 1, 49]])

# Count Vector

In [95]:
count_vectorizer = CountVectorizer()
count_data_train = count_vectorizer.fit_transform(X_train)
count_data_test = count_vectorizer.transform(X_test)

In [97]:
count_data_train.shape

(406, 9516)

## Naive Baise

In [98]:
nb_classifier.fit(count_data_train, y_train)
nb_train_preds = nb_classifier.predict(count_data_train)
nb_test_preds = nb_classifier.predict(count_data_test)

In [99]:
nb_train_score = accuracy_score(y_train, nb_train_preds)
nb_test_score = accuracy_score(y_test, nb_test_preds)
print ("NB")
print("Training Accuracy: {:.4} \t\t Testing Accuracy: {:.4}".format(nb_train_score, nb_test_score))
print("")
print('-'*70)

NB
Training Accuracy: 0.9951 		 Testing Accuracy: 0.9608

----------------------------------------------------------------------


In [100]:
confusion = metrics.confusion_matrix(y_test, nb_test_preds, labels = ['D','R'])
confusion

array([[48,  4],
       [ 0, 50]])

## Random Forest

In [101]:
rf_classifier.fit(count_data_train, y_train)
rf_train_preds = rf_classifier.predict(count_data_train)
rf_test_preds = rf_classifier.predict(count_data_test)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished
[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:    0.0s finished


In [102]:
rf_train_score = accuracy_score(y_train, rf_train_preds)
rf_test_score = accuracy_score(y_test, rf_test_preds)
print('Random Forest')
print("Training Accuracy: {:.4} \t\t Testing Accuracy: {:.4}".format(rf_train_score, rf_test_score))

Random Forest
Training Accuracy: 1.0 		 Testing Accuracy: 0.9118


In [103]:
confusion = metrics.confusion_matrix(y_test, rf_test_preds, labels = ['D','R'])
confusion

array([[50,  2],
       [ 7, 43]])

## XGboost

In [104]:
xgb_classifier.fit(count_data_train, y_train)
xgb_train_preds = xgb_classifier.predict(count_data_train)
xgb_test_preds = xgb_classifier.predict(count_data_test)

In [105]:
xgb_train_score = accuracy_score(y_train, xgb_train_preds)
xgb_test_score = accuracy_score(y_test, xgb_test_preds)
print('xgboost')
print("Training Accuracy: {:.4} \t\t Testing Accuracy: {:.4}".format(xgb_train_score, xgb_test_score))

xgboost
Training Accuracy: 1.0 		 Testing Accuracy: 0.951


In [106]:
confusion = metrics.confusion_matrix(y_test, xgb_test_preds, labels = ['D','R'])
confusion

array([[49,  3],
       [ 2, 48]])

In [117]:
df5 = pd.DataFrame(y_test)
df6 = pd.DataFrame(xgb_test_preds, columns = ['PRED'])
df6.head()

,PRED
0,R
1,R
2,R
3,D
4,D


In [120]:
df5['PRED'] = df6['PRED'].values

In [125]:
df5.reset_index(inplace = True)

In [129]:
for i in range(len(df5)):
    if df5.Target[i] != df5.PRED[i]:
        print (df5.username[i],df5.Target[i])

repjoshharder D
repvisclosky D
repbenmcadams D
repjohncurtis R
susanwbrooks R


In [132]:
import matplotlib.pyplot as plt
%matplotlib inline

In [145]:
df_count = pd.DataFrame(count_data_train.todense(),columns = count_vectorizer.get_feature_names())
feature_importance = pd.DataFrame(xgb_classifier.feature_importances_, index=df_count.columns, columns=['importance']).sort_values('importance',ascending=False)
feature_importance.head(50)

,importance
worker,0.060827
equalityact,0.043796
trumpshutdown,0.036496
climat,0.026764
news,0.024331
due,0.024331
forthepeopl,0.021898
life,0.021898
whitehous,0.019465
gender,0.019465


In [138]:
sorted_idx = np.argsort(xgb_classifier.feature_importances_)[::-1]

In [140]:
for index in sorted_idx:
    print([count_data_train.columns[index], xgb_classifier.feature_importances_[index]]) 

AttributeError: columns not found